In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
# read it in to inspect it
with open('Friends_Transcript.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(type(text))

<class 'str'>


In [5]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  4899189


In [6]:
print(text[:1000])

THE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)
Written by: Marta Kauffman & David Crane
[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]
Monica: There's nothing to tell! He's just some guy I work with!
Joey: C'mon, you're going out with the guy! There's gotta be something wrong with him!
Chandler: All right Joey, be nice. So does he have a hump? A hump and a hairpiece?
Phoebe: Wait, does he eat chalk?
(They all stare, bemused.)
Phoebe: Just, 'cause, I don't want her to go through what I went through with Carl- oh!
Monica: Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex.
Chandler: Sounds like a date to me.
[Time Lapse]
Chandler: Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked.
All: Oh, yeah. Had that dream.
Chandler: Then I look down, and I realize there's a phone... there.
Joey: Instead of...?
Chandler: That's right.


In [7]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_`abcdefghijklmnopqrstuvwxyz{|}
94


In [8]:
for i, x in enumerate(['d', 'f', 'gg']):
    print(i,x)

0 d
1 f
2 gg


In [9]:
## Tokenisation strategy from characters to integers

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
## saglya char la numbers deto
itos = { i:ch for i,ch in enumerate(chars) }

## two way dictionary(jugaad hai. 2 dictionary banayi hai)
## s t0 i
## i to s

encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[72, 73, 73, 1, 84, 72, 69, 82, 69]
hii there


In [10]:
## Sentencepeice is anotherway to encode text to integers. it is a subword tokeniser
## tiktoken is a tokeniser ised by chatgpt
## tiktoken has 50000 tokens.

In [11]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([4899189]) torch.int64
tensor([53, 41, 38,  1, 48, 47, 38,  1, 56, 41, 38, 51, 38,  1, 46, 48, 47, 42,
        36, 34,  1, 40, 38, 53, 52,  1, 34,  1, 47, 38, 56,  1, 51, 48, 48, 46,
        34, 53, 38,  1,  9, 53, 41, 38,  1, 49, 42, 45, 48, 53, 14, 53, 41, 38,
         1, 54, 47, 36, 54, 53,  1, 55, 38, 51, 52, 42, 48, 47, 10,  0, 56, 82,
        73, 84, 84, 69, 78,  1, 66, 89, 27,  1, 46, 65, 82, 84, 65,  1, 44, 65,
        85, 70, 70, 77, 65, 78,  1,  7,  1, 37, 65, 86, 73, 68,  1, 36, 82, 65,
        78, 69,  0, 60, 52, 67, 69, 78, 69, 27,  1, 36, 69, 78, 84, 82, 65, 76,
         1, 49, 69, 82, 75, 13,  1, 36, 72, 65, 78, 68, 76, 69, 82, 13,  1, 43,
        79, 69, 89, 13,  1, 49, 72, 79, 69, 66, 69, 13,  1, 65, 78, 68,  1, 46,
        79, 78, 73, 67, 65,  1, 65, 82, 69,  1, 84, 72, 69, 82, 69, 15, 61,  0,
        46, 79, 78, 73, 67, 65, 27,  1, 53, 72, 69, 82, 69,  8, 83,  1, 78, 79,
        84, 72, 73, 78, 71,  1, 84, 79,  1, 84, 69, 76, 76,  2,  1, 41, 69,  8,
      

In [12]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [13]:
## cant train on the entire thing at once. 
## so we make chunks
## their size is called block size

In [14]:
block_size = 8
train_data[:block_size+1]

tensor([53, 41, 38,  1, 48, 47, 38,  1, 56])

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([53]) the target: 41
when input is tensor([53, 41]) the target: 38
when input is tensor([53, 41, 38]) the target: 1
when input is tensor([53, 41, 38,  1]) the target: 48
when input is tensor([53, 41, 38,  1, 48]) the target: 47
when input is tensor([53, 41, 38,  1, 48, 47]) the target: 38
when input is tensor([53, 41, 38,  1, 48, 47, 38]) the target: 1
when input is tensor([53, 41, 38,  1, 48, 47, 38,  1]) the target: 56


In [16]:
## our transformer must be abe to predict next characters based on a given sequence of
## 1, 2, 3,.... or block_size=8 characters.
## like for c1 -> cx
## for c1,c2 -> cy
##
## for c1,c2,c3,c4,c5,c6,c7,c8 -> cz

In [17]:
## now, we want to process more than one such blocks at once parallely. 

## so we will have parallel channels where each will be working on a block with characters = blocksize


In [18]:
## batch size is how many sequences we are processing in parallel
## and block size is the number of characters per block

In [19]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

In [20]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    
    ## generate batchsize number of random offsets
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    ## x has the ith element and block size amt of elements following it.
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


In [21]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

inputs:
torch.Size([4, 8])
tensor([[79, 85,  1, 72, 65, 68,  1, 84],
        [78, 75,  1, 79, 70,  1, 84, 72],
        [65, 82, 69,  1, 89, 79, 85,  1],
        [78, 79, 13,  1, 83, 69, 69, 13]])
targets:
torch.Size([4, 8])
tensor([[85,  1, 72, 65, 68,  1, 84, 79],
        [75,  1, 79, 70,  1, 84, 72, 65],
        [82, 69,  1, 89, 79, 85,  1, 71],
        [79, 13,  1, 83, 69, 69, 13,  1]])
----


In [22]:
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

when input is [79] the target: 85
when input is [79, 85] the target: 1
when input is [79, 85, 1] the target: 72
when input is [79, 85, 1, 72] the target: 65
when input is [79, 85, 1, 72, 65] the target: 68
when input is [79, 85, 1, 72, 65, 68] the target: 1
when input is [79, 85, 1, 72, 65, 68, 1] the target: 84
when input is [79, 85, 1, 72, 65, 68, 1, 84] the target: 79
when input is [78] the target: 75
when input is [78, 75] the target: 1
when input is [78, 75, 1] the target: 79
when input is [78, 75, 1, 79] the target: 70
when input is [78, 75, 1, 79, 70] the target: 1
when input is [78, 75, 1, 79, 70, 1] the target: 84
when input is [78, 75, 1, 79, 70, 1, 84] the target: 72
when input is [78, 75, 1, 79, 70, 1, 84, 72] the target: 65
when input is [65] the target: 82
when input is [65, 82] the target: 69
when input is [65, 82, 69] the target: 1
when input is [65, 82, 69, 1] the target: 89
when input is [65, 82, 69, 1, 89] the target: 79
when input is [65, 82, 69, 1, 89, 79] the targ

In [45]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([512, 94])
tensor(5.0315, grad_fn=<NllLossBackward0>)

m"";/+,F(&MGF|kxaeuJ) Qrsti^?z6'0J<'rRW@yl3Esf}q]x<A`/Q3yCMlXF3+r`& WSDj-hyb_QO3hV{dO3I]VTNLt:e#L(iN


In [24]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [25]:
batch_size = 32
for steps in range(100): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


5.004701614379883


In [26]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


'}2;!]XYwJEot:@wB5a B7"GWp/1L `ac B9B$ eEN2tY6^A>_Q/PB%S/'>9}3H+Lb
.FWa9Y]_IL^K$b:^.X@qYEX,91B6!S,];pRpPr7<+%>y1^bF2B6(;(BM*d+.lhUv,EvEa%+&
@mb.;[Z9rQc?6Sg|j*-|&&uj-zVSLyts03V-08e!W=3rR}-D^,bVPZO,%aoPj$i9Nt&De$1Pae)I+m-_0! k%>(ghEhM-/VFxRW@I?kO/R-FTCi,Rj@H*;=c)P7A=!-gW/t|Ff Z=4{5Nkh+B|MWWQhgB$-SVEnbfIJpx6t8/t5j5!v7xRsty@UnJ
T!&2SzCKF@,I:Dsts}2m0'c{%Szy0Istsx(dz=bk E>BRlw"p:hi,K(B_}g*Q>8Gz^?FIGM/muDsti+L*<o`"Z9vB}+t%>9}L?jzXqj#LcAYfa{k_Q+v{{8zt
A/Ei@v6.V<u@mKr(bdw
^?|pHrVc-gBK]xAjd>A/]s Q.f&&WsNs


In [27]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [28]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [29]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [30]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [31]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [32]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [33]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [34]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [35]:
k.var()
q.var()
wei.var()

tensor(1.0918)

In [36]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [37]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [38]:
class LayerNorm1d: # (used to be BatchNorm1d)
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [39]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [40]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-3.5763e-09), tensor(1.0000))

In [52]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 1000
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 8
n_layer = 6
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('Friends_Transcript.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))


0.313054 M parameters
step 0: train loss 4.7428, val loss 4.7465
step 1000: train loss 1.8986, val loss 1.9184
step 2000: train loss 1.6969, val loss 1.7188
step 3000: train loss 1.6001, val loss 1.6368
step 4000: train loss 1.5415, val loss 1.5878
step 5000: train loss 1.5161, val loss 1.5550
step 6000: train loss 1.4787, val loss 1.5349
step 7000: train loss 1.4724, val loss 1.5189
step 8000: train loss 1.4390, val loss 1.4924
step 9000: train loss 1.4369, val loss 1.4851
step 9999: train loss 1.4084, val loss 1.4585

(She into on the door.) "Ross, laugh I'll think ready to get ball beir off, would!
Joey: Oh, oh, okay,?!
Chandler: Monica, ure we and he sees for the tick apartment.]
Janice: You got, I dinn two even they come their leady/fomiting!!
[Scene: Monica and Clearies are Finish, What you shome about him. Clea-line. All no killy wall (her girls to 537...' exctually know Chandler, yknow is maid about right.
Joey: Something!!
Him: Oh. Oh, you mistend women! I dont tell it with as

In [53]:
max_iters = 40000

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))


step 0: train loss 1.4220, val loss 1.4670
step 1000: train loss 1.4036, val loss 1.4410
step 2000: train loss 1.3991, val loss 1.4416
step 3000: train loss 1.3855, val loss 1.4317
step 4000: train loss 1.3796, val loss 1.4279
step 5000: train loss 1.3731, val loss 1.4274
step 6000: train loss 1.3656, val loss 1.4363
step 7000: train loss 1.3578, val loss 1.4079
step 8000: train loss 1.3566, val loss 1.4032
step 9000: train loss 1.3438, val loss 1.4036
step 10000: train loss 1.3495, val loss 1.3977
step 11000: train loss 1.3345, val loss 1.4138
step 12000: train loss 1.3494, val loss 1.4013
step 13000: train loss 1.3405, val loss 1.3995
step 14000: train loss 1.3404, val loss 1.3964
step 15000: train loss 1.3338, val loss 1.3981
step 16000: train loss 1.3300, val loss 1.3845
step 17000: train loss 1.3317, val loss 1.3926
step 18000: train loss 1.3136, val loss 1.3752
step 19000: train loss 1.3192, val loss 1.3908
step 20000: train loss 1.3152, val loss 1.3781
step 21000: train loss 1.3